## **Chapter 4. Using Examples and Few-Shot Prompting** 

There are a number of well-known prompting techniques that use examples, such as zero-shot, one-shot, and few-shot prompting. By understanding each approach and the principles of designing effective examples, you can significantly enhance the performance of Solar Pro.

- [zero-shot prompting](#zeroshot)
- [one-shot prompting](#oneshot)
- [few-shot prompting](#fewshot)

**Set up**

In [1]:
from openai import OpenAI

# Retrieve the UPSTAGE_API_KEY variable from the IPython store
%store -r UPSTAGE_API_KEY

client = OpenAI(
    api_key= UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

config_model = {
    "model": "solar-pro",
    "max_tokens": 500,
    "temperature": 0.7,
    "top_p": 0.9,
}

---

<a id="zeroshot"></a>
### **4.1 Zero-shot prompting**

A technique where the model is asked to perform a task without any prior examples. There are tons of ways to craft zero-shot prompting. 

#### **Content Example**

In [2]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "Classify the following text as positive, negative, or neutral. Text: I thought the macaron flavor was just okay. Sentiment: { }"
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)


Sentiment: neutral


#### **Practice Exercise**

→ Translate the term 'artificial tears' from English into Japanese. 

In [ ]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "<<[ Replace this text ]>>"
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)


<a id="oneshot"></a>
### **4.2 One-shot prompting**

One-shot prompting is a technique where the model is provided with only one example before it’s asked to complete the task.

#### **Content Example**

In [4]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "Translate the following sentence into French: I like reading books."
    },
    {
        "role":"assistant",
        "content": "J'aime lire des livres."
    },
    {
        "role":"user",
        "content": "Translate the following sentence into French: I love studying language"
    },
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

J'adore étudier les langues.


#### **Practice Exercise**

→ Translate the term 'artificial tears' from English to Japanese using one-shot prompting.

In [ ]:
# Prompt:
message = [{"<<[ Replace this text ]>>"}]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

<a id="fewshot"></a>
### **4.3 Few-shot Prompting**

Few-shot prompting is a technique where the model is given a handful of examples—typically two to five—before it is asked to complete the task. This approach offers the model multiple references for the type of output desired, providing more context and making the output more accurate or in line with specific patterns.

> **It's important to note that few-shot learning and few-shot prompting are distinct concepts. Few-shot learning is a broader machine learning approach focused on adapting model parameters with only a few examples. In contrast, few-shot prompting specifically applies to prompt design in generative AI, where model parameters remain unchanged.**

#### **Content Examples**

In [9]:
# Prompt:
message = [
    {
        "role":"user",
        "content": "2+10: twelve, 4+52: fifty-six, 100+301: "
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

four hundred and one


We would like to highlight four ways to design effective examples when using Solar Pro. 

- [**exemplar quantity**](#quant)
- [**exemplar similarity**](#sim)
- [**exemplar format**](#format)

<a id="quant"></a>
##### (1) Exemplar Quantity

Mostly, the quantity of exemplars in the prompt generally improves model
performance. 

In [10]:
# Prompt:
message = [
    {
        "role":"user",
        "content": """
Sentence: "The cat sleeps."
Subject: "The cat"
Verb: "sleeps"
Object: None
Analysis: Simple subject-verb sentence structure with a single actor performing an action.

##
Sentence: "She reads books."
Subject: "She"
Verb: "reads"
Object: "books"
Analysis: Subject-verb-object structure, indicating the action is directed toward an object.

## 
Sentence: "They dance gracefully."
Subject: "They"
Verb: "dance"
Adverb: "gracefully"
Analysis: Subject-verb-adverb structure, with the adverb modifying how the action is performed.

## 
Sentence: "The sun rises in the east."
Subject: "The sun"
Verb: "rises"
Prepositional Phrase: "in the east"
Analysis: Subject-verb-prepositional phrase structure, adding context about the action's location.

##
setence: Thick smog chokes northern India and eastern Pakistan ahead of Diwali.
"""
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

Sentence: "Thick smog chokes northern India and eastern Pakistan ahead of Diwali."
Subject: "Thick smog"
Verb: "chokes"
Object: "northern India and eastern Pakistan"
Adverbial Phrase: "ahead of Diwali"
Analysis: Subject-verb-object-adverbial phrase structure, indicating the action is directed toward an object and providing context about the action's timing.


In [11]:
# without an example

# Prompt:
message = [
    {
        "role":"user",
        "content": """Please analyze and define the sentence components in the following sentence.
        sentence: "Thick smog chokes northern India and eastern Pakistan ahead of Diwali."
        """
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

Subject: Thick smog
Verb: chokes
Object 1: northern India
Object 2: eastern Pakistan
Adverb: ahead of Diwali


- Observation: If you provide an example, you can see the model respond in the format used in the example. Even with complex and varied sentence structures, using many examples to guide the model helps it perform the task well. 

<a id="sim"></a>
##### (2) Exemplar Similarity

Select exemplars that are similar to your task. For example, if you are summarizing a news article, using exemplars in the same format will yield the best results you desire.

**content example**

In the following prompt example, you can see that the summary results use **a consistent response format** as an example. By increasing the consistency of the format, you can achieve more accurate results.

In [28]:
# When examples have the similarities
# Prompt:
message = [
    {
        "role":"user",
        "content": """Read the following text and summarize the key points. Once you finish the summary, organize it as follows:

#Keyword 1 / Related key point in one short phrase./
#Keyword 2 / Related key point in one short phrase./
#Keyword 3 / Related key point in one short phrase./

<text>
To be considered open source under the OSAID, an AI model has to provide enough information about its design so that a person could “substantially” recreate it. The model must also disclose any pertinent details about its training data, including the provenance, how the data was processed, and how it can be obtained or licensed.
“An open source AI is an AI model that allows you to fully understand how it’s been built,” Maffulli said. “That means that you have access to all the components, such as the complete code used for training and data filtering.”
The OSAID also lays out usage rights developers should expect with open source AI, like the freedom to use the model for any purpose and modify it without having to ask anyone’s permission. “Most importantly, you should be able to build on top,” added Maffulli.
The OSI has no enforcement mechanisms to speak of. It can’t pressure developers to abide by or follow the OSAID. But it does intend to flag models described as “open source” but which fall short of the definition.
“Our hope is that when someone tries to abuse the term, the AI community will say, ‘We don’t recognize this as open source,’ and it gets corrected,” Maffulli said. Historically, this has had mixed results, but it isn’t entirely without effect.
Many startups and big tech companies, most prominently Meta, have employed the term “open source” to describe their AI model release strategies — but few meet the OSAID’s criteria. For example, Meta mandates that platforms with over 700 million monthly active users request a special license to use its [Llama](https://techcrunch.com/2024/09/08/meta-llama-everything-you-need-to-know-about-the-open-generative-ai-model/) models.
Maffulli has been [openly critical](https://www.ft.com/content/397c50d8-8796-4042-a814-0ac2c068361f) of Meta’s decision to call its models “open source.” After discussions with the OSI, Google and Microsoft agreed to drop their use of the term for models that aren’t fully open, but Meta hasn’t, he said.
Stability AI, which has long advertised its models as “open,” requires that businesses making more than $1 million in revenue obtain an enterprise license. And French AI upstart Mistral’s license bars the use of certain models and outputs for commercial ventures.
A [study](https://www.wired.com/story/the-myth-of-open-source-ai/) last August by researchers at the Signal Foundation, the nonprofit AI Now Institute, and Carnegie Mellon found that many “open source” models are basically open source in name only. The data required to train the models is kept secret, the compute power needed to run them is beyond the reach of many developers, and the techniques to fine-tune them are intimidatingly complex.
Instead of democratizing AI, these “open source” projects tend to entrench and expand centralized power, the study’s authors concluded. Indeed, Meta’s Lllama models have [racked up](https://venturebeat.com/ai/meta-leads-open-source-ai-boom-llama-downloads-surge-10x-year-over-year/) hundreds of millions of downloads, and Stability [claims](https://www.prnewswire.com/news-releases/stability-ai-secures-significant-new-investment-from-world-class-investor-group-and-appoints-prem-akkaraju-as-ceo-302181923.html) that its models power up to 80% of all AI-generated imagery.
</text>"""
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

#Open Source AI Definition / AI model with sufficient design information for substantial recreation, disclosed training data details./
#OSAID Key Points / Freedom to use, modify, and build on top of open source AI models./
#Challenges and Misuse / Mixed results in enforcing open source AI definition, with some companies not meeting criteria or misusing the term./


In [37]:
# When examples don't have the similarities
# Prompt:
message = [
    {
        "role":"user",
        "content": """Read the following text and summarize the key points. Once you finish the summary, organize it as follows:

Related key point in one short phrase./Keyword 1
Keyword 2 Related key point in two short phrases.
Keyword 3/ Related key point in three short phrases.

<text>
To be considered open source under the OSAID, an AI model has to provide enough information about its design so that a person could “substantially” recreate it. The model must also disclose any pertinent details about its training data, including the provenance, how the data was processed, and how it can be obtained or licensed.
“An open source AI is an AI model that allows you to fully understand how it’s been built,” Maffulli said. “That means that you have access to all the components, such as the complete code used for training and data filtering.”
The OSAID also lays out usage rights developers should expect with open source AI, like the freedom to use the model for any purpose and modify it without having to ask anyone’s permission. “Most importantly, you should be able to build on top,” added Maffulli.
The OSI has no enforcement mechanisms to speak of. It can’t pressure developers to abide by or follow the OSAID. But it does intend to flag models described as “open source” but which fall short of the definition.
“Our hope is that when someone tries to abuse the term, the AI community will say, ‘We don’t recognize this as open source,’ and it gets corrected,” Maffulli said. Historically, this has had mixed results, but it isn’t entirely without effect.
Many startups and big tech companies, most prominently Meta, have employed the term “open source” to describe their AI model release strategies — but few meet the OSAID’s criteria. For example, Meta mandates that platforms with over 700 million monthly active users request a special license to use its [Llama](https://techcrunch.com/2024/09/08/meta-llama-everything-you-need-to-know-about-the-open-generative-ai-model/) models.
Maffulli has been [openly critical](https://www.ft.com/content/397c50d8-8796-4042-a814-0ac2c068361f) of Meta’s decision to call its models “open source.” After discussions with the OSI, Google and Microsoft agreed to drop their use of the term for models that aren’t fully open, but Meta hasn’t, he said.
Stability AI, which has long advertised its models as “open,” requires that businesses making more than $1 million in revenue obtain an enterprise license. And French AI upstart Mistral’s license bars the use of certain models and outputs for commercial ventures.
A [study](https://www.wired.com/story/the-myth-of-open-source-ai/) last August by researchers at the Signal Foundation, the nonprofit AI Now Institute, and Carnegie Mellon found that many “open source” models are basically open source in name only. The data required to train the models is kept secret, the compute power needed to run them is beyond the reach of many developers, and the techniques to fine-tune them are intimidatingly complex.
Instead of democratizing AI, these “open source” projects tend to entrench and expand centralized power, the study’s authors concluded. Indeed, Meta’s Lllama models have [racked up](https://venturebeat.com/ai/meta-leads-open-source-ai-boom-llama-downloads-surge-10x-year-over-year/) hundreds of millions of downloads, and Stability [claims](https://www.prnewswire.com/news-releases/stability-ai-secures-significant-new-investment-from-world-class-investor-group-and-appoints-prem-akkaraju-as-ceo-302181923.html) that its models power up to 80% of all AI-generated imagery.

</text>"""
    }
]

config = {**config_model, "messages": message}

responses = []
for i in range(3):
    response = client.chat.completions.create(**config)
    responses.append(response.choices[0].message.content)

- As seen in the following results, prompts with lower similarity among examples yield a wider variety of outcomes.

In [57]:
for i in range(3):
    print(f"n = {i+1}")
    print(responses[i])
    print("\n")

n = 1
Keyword 1: Open Source AI Definition
Open source AI requires detailed design and training data information.

Keyword 2: Usage Rights and Community Enforcement
Open source AI allows full usage, modification, and building upon; community enforcement relies on recognition.

Keyword 3: Industry Practices and Criticism
Many companies' AI models, despite being labeled "open source," do not meet the definition, leading to centralized power and criticism from the AI community.


n = 2
Key point: Open source AI definition
OSAID defines open source AI as a model with accessible design and training data information.

Keyword 1: Open source AI
Related key point: Full understanding and usage rights
An open source AI allows full understanding of its construction and grants freedom to use, modify, and build upon it.

Keyword 2: OSAID enforcement
Related key point: Community pressure and mixed results
The OSI has no enforcement mechanisms but relies on community pressure to correct misuse of the

<a id="format"></a>
##### (3) Exemplar Format

Use a common template. Optimal format may vary across tasks. 

There is evidence indicating that formats frequently found in the training data tend to result in improved performance.

**Content Example**

In [70]:
# Prompt:
message = [
    {
        "role":"user",
        "content": """Identify the entities referenced in the text below. 

Please extract the following four types of entities:
- company names: 
- personal names: 
- specific topics:
- themes:

<text>
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efficiently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently. We show experimentally that DUS is simple yet effective in scaling up high-performance LLMs from small ones. Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, a variant fine-tuned for instruction-following capabilities, surpassing Mixtral-8x7B-Instruct. SOLAR 10.7B is publicly available under the Apache 2.0 license, promoting broad access and application in the LLM field.
</text>"""
    }
]

config = {**config_model, "messages": message}

response = client.chat.completions.create(**config)
print(response.choices[0].message.content)

company names:
- SOLAR

personal names:
- None

specific topics:
- large language models (LLMs)
- depth up-scaling (DUS)
- mixture-of-experts
- Mixtral-8x7B-Instruct
- Apache 2.0 license

themes:
- natural language processing (NLP) tasks
- efficient up-scaling of LLMs
- depthwise scaling
- continued pretraining
- instruction-following capabilities


In [71]:
# When not using a standard format
# Prompt:
message = [
    {
        "role":"user",
        "content": """Identify the entities referenced in the text below. 

Please extract the following four types of entities:
% % company names
% % personal names 
% % specific topics% % 
% %  themes % % 

~text~
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efficiently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently. We show experimentally that DUS is simple yet effective in scaling up high-performance LLMs from small ones. Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, a variant fine-tuned for instruction-following capabilities, surpassing Mixtral-8x7B-Instruct. SOLAR 10.7B is publicly available under the Apache 2.0 license, promoting broad access and application in the LLM field.
~text~"""
    }
]

config = {**config_model, "messages": message}

responses = []
for i in range(2):
    response = client.chat.completions.create(**config)
    responses.append(response.choices[0].message.content)

- When using unusual format symbols or marks, the stability of the results decreases each time they are generated.

In [72]:
for i in range(2):
    print(f"n = {i+1}")
    print(responses[i])
    print("\n")

n = 1
company names:
* (none)

personal names:
* (none)

specific topics:
* large language models (LLM)
* natural language processing (NLP) tasks
* depth up-scaling (DUS)
* depthwise scaling
* continued pretraining
* mixture-of-experts
* SOLAR 10.7B
* SOLAR 10.7B-Instruct
* Mixtral-8x7B-Instruct

themes:
* scaling up LLMs
* instruction-following capabilities
* public availability of LLMs


n = 2
company names:
* SOLAR (referring to the company that developed SOLAR 10.7B)

personal names:
* None

specific topics:
* large language model (LLM)
* natural language processing (NLP) tasks
* depth up-scaling (DUS)
* depthwise scaling
* continued pretraining
* mixture-of-experts
* instruction-following capabilities
* Apache 2.0 license

themes:
* scaling up large language models
* improving performance in natural language processing tasks
* efficient training and inference methods
* promoting broad access and application in the LLM field




#### **Practice Exercise**

**Well Done!**

Having completed all the exercises, you're now ready to proceed to the next chapter.